In [69]:
import pandas as pd # DataFrame management
from selenium import webdriver # WebScraping driver
from selenium.webdriver.common.by import By # WebScraping property library
import time # Sleep library

In [70]:
driver = webdriver.Firefox()

In [71]:
base_url = 'https://www.oddsportal.com/it/'
url2 = 'https://www.oddsportal.com/it/tennis/france/atp-french-open/'
driver.get(base_url)
time.sleep(1)
driver.find_element(By.ID, 'onetrust-accept-btn-handler').click()


In [77]:
buttons_list = driver.find_elements(By.CSS_SELECTOR, '.flex.flex-col.w-full.h-auto')
if len(buttons_list)==0:
    buttons_list = driver.find_elements(By.CSS_SELECTOR, '.text-white-main.flex.h-full.items-center a')

In [78]:
print(len(buttons_list))

21


In [89]:
sports_links_dict = {}
for button in buttons_list:
        href = button.get_attribute('href')
        sport = button.find_elements(By.CSS_SELECTOR, 'div')[1].text
        if len(sport)==0:
            sport = href.split('/')[-2]
        sports_links_dict[sport.capitalize()] = href


In [124]:
# for sport in sports_links_dict.keys():
comps_links_dict = {}
list_cat = []
driver.get(sports_links_dict['Calcio'])
time.sleep(1)
list_categories = driver.find_elements(By.CSS_SELECTOR, '.flex.items-center.w-full.h-10.gap-1.bg-gray-med_light span')
for elem in list_categories:
    list_cat.append(elem.text)
list_competitions = driver.find_elements(By.CSS_SELECTOR, 'main .flex.flex-col .flex ul')
list_comp = []
for comp in list_competitions:
    single_comp = []
    single = comp.find_elements(By.CSS_SELECTOR, 'li a')
    for elem in single:
        name = ' '.join(elem.text.split(' ')[:-1])
        single_comp.append({name: elem.get_attribute('href')})
    list_comp.append(single_comp)

for i, j in zip(list_cat, list_comp):
    comps_links_dict[i] = j


In [125]:
comps_links_dict

{'Popolari': [{'Europei 2024': 'https://www.oddsportal.com/it/football/europe/europei-2024/'},
  {'Europei U21': 'https://www.oddsportal.com/it/football/europe/campionati-europei-u21/'}],
 'Argentina': [{'Liga Profesional': 'https://www.oddsportal.com/it/football/argentina/liga-profesional/'},
  {'Primera Nacional': 'https://www.oddsportal.com/it/football/argentina/primera-nacional/'},
  {'Copa Argentina': 'https://www.oddsportal.com/it/football/argentina/copa-argentina/'}],
 'Australia': [{'NPL ACT': 'https://www.oddsportal.com/it/football/australia/npl-act/'},
  {'NPL Northern NSW': 'https://www.oddsportal.com/it/football/australia/npl-northern-nsw/'},
  {'NPL Queensland': 'https://www.oddsportal.com/it/football/australia/npl-queensland/'},
  {'NSW League One': 'https://www.oddsportal.com/it/football/australia/nsw-league-one/'},
  {'Queensland Premier League': 'https://www.oddsportal.com/it/football/australia/queensland-premier-league/'}],
 'Austria': [{'Burgenland': 'https://www.odd

In [40]:
all_events = driver.find_elements(By.CSS_SELECTOR, '.group.flex')
all_events.pop(0)

<selenium.webdriver.remote.webelement.WebElement (session="a5494b1d-da1b-4409-a50a-a142c4a3e957", element="202f08e9-7b7e-4f00-b8d5-8463501780e0")>

In [41]:
print(len(all_events))

4


In [42]:
list_events = []
for event in all_events:
    event_dict = {}
    names = event.find_elements(By.CSS_SELECTOR, 'p')
    event_dict['Time'] = names[0].text
    event_dict['Player1'] = names[1].text
    event_dict['Player2'] = names[2].text
    event_dict['1'] = names[3].text
    event_dict['2'] = names[4].text
    list_events.append(event_dict)

In [44]:
df = pd.DataFrame(list_events)

In [45]:
df

,Time,Player1,Player2,1,2
0,14:00,Dimitrov G.,Sinner J.,4.51,1.20
1,20:15,Tsitsipas S.,Alcaraz C.,4.23,1.23
2,11:00,Djokovic N.,Ruud C.,2.00,1.80
3,11:00,Zverev A.,De Minaur A.,1.40,2.93
